In [1]:
# ============================================================================
# IMPORTS
# ============================================================================
import pandas as pd
import numpy as np
import time
import pickle
from collections import Counter

# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.compose import ColumnTransformer

# Gestion du déséquilibre
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTETomek

# Modèles
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

# Métriques
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, 
    f1_score, roc_auc_score, confusion_matrix,
    classification_report
)

import warnings
warnings.filterwarnings('ignore')



In [2]:

# ============================================================================
# 1. IMPORTATION DE LA BASE
# ============================================================================
def load_data(filepath):
    """Charge les données depuis un fichier CSV."""
    print("Chargement des données...")
    df = pd.read_csv(filepath)
    print(f"Dimensions : {df.shape}")
    print(f"Valeurs manquantes : {df.isnull().sum().sum()}")
    return df



In [3]:

# ============================================================================
# 2. SPLIT TRAIN/TEST
# ============================================================================
def split_data(df, target_column, test_size=0.2, random_state=42):
    """Sépare les données en ensembles d'entraînement et de test."""
    print("\nSéparation des données...")
    
    X = df.drop(columns=[target_column])
    y = df[target_column]
    
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_state, stratify=y
    )
    
    print(f"Train : {X_train.shape[0]} échantillons")
    print(f"Test : {X_test.shape[0]} échantillons")
    print(f"Distribution train : {Counter(y_train)}")
    
    return X_train, X_test, y_train, y_test


In [4]:


# ============================================================================
# 3. GESTION DES VALEURS MANQUANTES
# ============================================================================

# 3.1 Fonction d'imputation
def impute_missing_values(X_train, X_test, method="median"):
    """
    Impute les valeurs manquantes.
    
    Parameters:
    -----------
    method : str
        "median", "mean", "knn"
    """
    print(f"\nImputation avec méthode : {method}")
    
    X_train = X_train.copy()
    X_test = X_test.copy()
    
    # Conversion des booléens
    bool_cols = X_train.select_dtypes(include=["bool"]).columns
    X_train[bool_cols] = X_train[bool_cols].astype(int)
    X_test[bool_cols] = X_test[bool_cols].astype(int)
    
    # Détection des types
    num_cols = X_train.select_dtypes(include=["int64", "float64"]).columns
    cat_cols = X_train.select_dtypes(include=["object"]).columns
    
    # Imputers
    if method == "knn":
        num_imputer = KNNImputer(n_neighbors=5)
    elif method == "mean":
        num_imputer = SimpleImputer(strategy="mean")
    else:
        num_imputer = SimpleImputer(strategy="median")
    
    cat_imputer = SimpleImputer(strategy="most_frequent")
    
    preprocessor = ColumnTransformer(
        transformers=[
            ("num", num_imputer, num_cols),
            ("cat", cat_imputer, cat_cols)
        ],
        remainder="drop"
    )
    
    X_train_clean = preprocessor.fit_transform(X_train)
    X_test_clean = preprocessor.transform(X_test)
    
    print(f"Shape après imputation - Train: {X_train_clean.shape}, Test: {X_test_clean.shape}")
    
    return X_train_clean, X_test_clean


# 3.2 Fonction de suppression
def drop_missing_values(X_train, X_test, y_train, y_test):
    """Supprime les lignes contenant des valeurs manquantes."""
    print("\nSuppression des valeurs manquantes...")
    
    X_train = X_train.copy()
    X_test = X_test.copy()
    
    # Train
    mask_train = ~X_train.isna().any(axis=1)
    X_train_clean = X_train[mask_train].reset_index(drop=True)
    y_train_clean = y_train[mask_train].reset_index(drop=True)
    
    # Test
    mask_test = ~X_test.isna().any(axis=1)
    X_test_clean = X_test[mask_test].reset_index(drop=True)
    y_test_clean = y_test[mask_test].reset_index(drop=True)
    
    print(f"Train : {len(X_train)} → {len(X_train_clean)} ({len(X_train) - len(X_train_clean)} supprimés)")
    print(f"Test : {len(X_test)} → {len(X_test_clean)} ({len(X_test) - len(X_test_clean)} supprimés)")
    
    return X_train_clean, X_test_clean, y_train_clean, y_test_clean



In [5]:

# ============================================================================
# 4. GESTION DES DÉSÉQUILIBRES
# ============================================================================
def handle_imbalance(X_train, y_train, method="none"):
    """
    Gère le déséquilibre des classes.
    
    Parameters:
    -----------
    method : str
        "none", "weight", "smote", "smote_tomek"
    """
    print(f"\n--- Gestion du déséquilibre : {method.upper()} ---")
    print(f"Distribution avant : {Counter(y_train)}")
    
    if method == "none":
        return X_train, y_train, None
    
    elif method == "weight":
        n_samples = len(y_train)
        n_classes = len(np.unique(y_train))
        class_weight = {
            cls: n_samples / (n_classes * count) 
            for cls, count in Counter(y_train).items()
        }
        print(f"Poids calculés : {class_weight}")
        return X_train, y_train, class_weight
    
    elif method == "smote":
        smote = SMOTE(random_state=42, k_neighbors=5)
        X_resampled, y_resampled = smote.fit_resample(X_train, y_train)
        print(f"Distribution après SMOTE : {Counter(y_resampled)}")
        return X_resampled, y_resampled, None
    
    elif method == "smote_tomek":
        smote_tomek = SMOTETomek(random_state=42)
        X_resampled, y_resampled = smote_tomek.fit_resample(X_train, y_train)
        print(f"Distribution après SMOTE+Tomek : {Counter(y_resampled)}")
        return X_resampled, y_resampled, None
    
    else:
        raise ValueError(f"Méthode inconnue : {method}")



In [6]:

# ============================================================================
# 5. CONFIGURATION DES MODÈLES
# ============================================================================
def get_models(class_weight=None, scale_pos_weight=1):
    """Retourne les trois modèles configurés."""
    
    models = {
        'KNN': KNeighborsClassifier(
            n_neighbors=5,
            metric='minkowski',
            p=2
        ),
        'Random_Forest': RandomForestClassifier(
            n_estimators=100,
            random_state=42,
            class_weight=class_weight,
            max_depth=None,
            min_samples_split=2
        ),
        'XGBoost': XGBClassifier(
            random_state=42,
            scale_pos_weight=scale_pos_weight,
            use_label_encoder=False,
            eval_metric='logloss',
            n_estimators=100,
            learning_rate=0.1
        )
    }
    
    return models



In [7]:

# ============================================================================
# 6. ENTRAÎNEMENT
# ============================================================================
def train_models(models, X_train, y_train):
    """Entraîne tous les modèles."""
    print("\n" + "="*70)
    print("ENTRAÎNEMENT DES MODÈLES")
    print("="*70)
    
    trained_models = {}
    
    for model_name, model in models.items():
        print(f"\n{model_name}...")
        start_time = time.time()
        
        model.fit(X_train, y_train)
        
        elapsed_time = time.time() - start_time
        trained_models[model_name] = {
            'model': model,
            'training_time': elapsed_time
        }
        
        print(f"Temps d'entraînement : {elapsed_time:.2f}s")
    
    return trained_models



In [8]:

# ============================================================================
# 7. ÉVALUATION
# ============================================================================
def evaluate_models(trained_models, X_train, y_train, X_test, y_test):
    """Évalue les modèles sur train et test."""
    print("\n" + "="*70)
    print("ÉVALUATION DES MODÈLES")
    print("="*70)
    
    results = {}
    
    for model_name, model_info in trained_models.items():
        print(f"\n{'='*70}")
        print(f"{model_name}")
        print(f"{'='*70}")
        
        model = model_info['model']
        
        # Prédictions
        y_train_pred = model.predict(X_train)
        y_test_pred = model.predict(X_test)
        
        y_train_proba = model.predict_proba(X_train)[:, 1]
        y_test_proba = model.predict_proba(X_test)[:, 1]
        
        # Métriques Train
        train_metrics = {
            'accuracy': accuracy_score(y_train, y_train_pred),
            'precision': precision_score(y_train, y_train_pred),
            'recall': recall_score(y_train, y_train_pred),
            'f1': f1_score(y_train, y_train_pred),
            'auc': roc_auc_score(y_train, y_train_proba)
        }
        
        # Métriques Test
        test_metrics = {
            'accuracy': accuracy_score(y_test, y_test_pred),
            'precision': precision_score(y_test, y_test_pred),
            'recall': recall_score(y_test, y_test_pred),
            'f1': f1_score(y_test, y_test_pred),
            'auc': roc_auc_score(y_test, y_test_proba)
        }
        
        # Affichage
        print("\nTRAIN:")
        for metric, value in train_metrics.items():
            print(f"  {metric.upper():12} : {value:.4f}")
        
        print("\nTEST:")
        for metric, value in test_metrics.items():
            print(f"  {metric.upper():12} : {value:.4f}")
        
        print("\nMatrice de confusion (Test):")
        print(confusion_matrix(y_test, y_test_pred))
        
        # Stockage
        results[model_name] = {
            'train_metrics': train_metrics,
            'test_metrics': test_metrics,
            'confusion_matrix': confusion_matrix(y_test, y_test_pred),
            'training_time': model_info['training_time']
        }
    
    return results



In [9]:
# ============================================================================
# 8. SAUVEGARDE DES MODÈLES ET RÉSULTATS
# ============================================================================
import os
import json

def save_models(trained_models, results, prefix="model"):
    """
    Sauvegarde les modèles et leurs résultats dans des dossiers séparés.
    
    Parameters:
    -----------
    trained_models : dict
        Dictionnaire contenant les modèles entraînés
    results : dict
        Dictionnaire contenant les résultats d'évaluation
    prefix : str
        Préfixe pour les noms de fichiers
    """
    print("\n" + "="*70)
    print("SAUVEGARDE DES MODÈLES ET RÉSULTATS")
    print("="*70)
    
    # Créer les dossiers s'ils n'existent pas
    models_dir = "../models"
    results_dir = "../results"
    
    os.makedirs(models_dir, exist_ok=True)
    os.makedirs(results_dir, exist_ok=True)
    
    print(f"\nDossier modèles : {models_dir}")
    print(f"Dossier résultats : {results_dir}")
    
    for model_name, model_info in trained_models.items():
        # Nom des fichiers
        model_filename = os.path.join(models_dir, f"{prefix}_{model_name}.pkl")
        results_filename = os.path.join(results_dir, f"{prefix}_{model_name}_results.json")
        
        # Sauvegarde du modèle
        with open(model_filename, 'wb') as f:
            pickle.dump(model_info['model'], f)
        print(f"✓ Modèle sauvegardé : {model_filename}")
        
        # Préparation des résultats pour JSON
        results_to_save = {
            'train_metrics': results[model_name]['train_metrics'],
            'test_metrics': results[model_name]['test_metrics'],
            'confusion_matrix': results[model_name]['confusion_matrix'].tolist(),
            'training_time': results[model_name]['training_time']
        }
        
        # Sauvegarde des résultats
        with open(results_filename, 'w') as f:
            json.dump(results_to_save, f, indent=4)
        print(f"✓ Résultats sauvegardés : {results_filename}")
    
    print("\n" + "="*70)
    print("Tous les modèles et résultats ont été sauvegardés.")
    print("="*70)

In [10]:

# ============================================================================
# 9. PIPELINE COMPLET
# ============================================================================
def run_pipeline(filepath, target_column, 
                 missing_method="impute", impute_strategy="median",
                 balance_method="smote", 
                 save_prefix="model"):
    """
    Exécute le pipeline complet de modélisation.
    
    Parameters:
    -----------
    filepath : str
        Chemin vers le fichier CSV
    target_column : str
        Nom de la colonne cible
    missing_method : str
        "impute" ou "drop"
    impute_strategy : str
        "median", "mean", "knn" (si missing_method="impute")
    balance_method : str
        "none", "weight", "smote", "smote_tomek"
    save_prefix : str
        Préfixe pour les fichiers sauvegardés
    """
    
    print("\n" + "="*70)
    print("PIPELINE DE MODÉLISATION")
    print("="*70)
    print(f"Fichier : {filepath}")
    print(f"Gestion des manquantes : {missing_method}")
    if missing_method == "impute":
        print(f"Stratégie d'imputation : {impute_strategy}")
    print(f"Gestion du déséquilibre : {balance_method}")
    print("="*70)
    
    # 1. Chargement
    df = load_data(filepath)
    
    # 2. Split
    X_train, X_test, y_train, y_test = split_data(df, target_column)
    
    # 3. Gestion des valeurs manquantes
    if missing_method == "impute":
        X_train_clean, X_test_clean = impute_missing_values(
            X_train, X_test, method=impute_strategy
        )
        y_train_clean = y_train
        y_test_clean = y_test
    elif missing_method == "drop":
        X_train_clean, X_test_clean, y_train_clean, y_test_clean = drop_missing_values(
            X_train, X_test, y_train, y_test
        )
    else:
        raise ValueError(f"missing_method inconnu : {missing_method}")
    
    # 4. Gestion du déséquilibre
    X_train_balanced, y_train_balanced, class_weight = handle_imbalance(
        X_train_clean, y_train_clean, method=balance_method
    )
    
    # Calcul du scale_pos_weight pour XGBoost
    if balance_method in ["none", "weight"]:
        ratio = Counter(y_train_clean)[0] / Counter(y_train_clean)[1]
    else:
        ratio = 1
    
    # 5. Configuration des modèles
    if balance_method == "weight":
        models = get_models(class_weight=class_weight, scale_pos_weight=ratio)
    elif balance_method in ["smote", "smote_tomek"]:
        models = get_models(class_weight=None, scale_pos_weight=1)
    else:
        models = get_models(class_weight=None, scale_pos_weight=ratio)
    
    # 6. Entraînement
    trained_models = train_models(models, X_train_balanced, y_train_balanced)
    
    # 7. Évaluation
    results = evaluate_models(
        trained_models, 
        X_train_balanced, y_train_balanced,
        X_test_clean, y_test_clean
    )
    
    # 8. Sauvegarde
    save_models(trained_models, results, prefix=save_prefix)
    
    return trained_models, results



In [ ]:

# ============================================================================
# 10. EXÉCUTION
# ============================================================================
if __name__ == "__main__":
    
    # Configuration
    FILEPATH = "../data/df_dropna_normalized.csv"
    TARGET = "embauche"  # Nom de votre colonne cible
    
    # Scénarios à tester
    scenarios = [
        {
            'name': 'Imputation_Median_SMOTE',
            'missing_method': 'impute',
            'impute_strategy': 'median',
            'balance_method': 'smote'
        },
        {
            'name': 'Imputation_KNN_SMOTE',
            'missing_method': 'impute',
            'impute_strategy': 'knn',
            'balance_method': 'smote'
        },
        {
            'name': 'Drop_SMOTE',
            'missing_method': 'drop',
            'impute_strategy': None,
            'balance_method': 'smote'
        },
        {
            'name': 'Imputation_Median_SMOTETomek',
            'missing_method': 'impute',
            'impute_strategy': 'median',
            'balance_method': 'smote_tomek'
        },
        {
            'name': 'Imputation_Median_Weight',
            'missing_method': 'impute',
            'impute_strategy': 'median',
            'balance_method': 'weight'
        }
    ]
    
    # Exécution de tous les scénarios
    all_results = {}
    
    for scenario in scenarios:
        print("\n\n" + "#"*70)
        print(f"# SCÉNARIO : {scenario['name']}")
        print("#"*70)
        
        trained_models, results = run_pipeline(
            filepath=FILEPATH,
            target_column=TARGET,
            missing_method=scenario['missing_method'],
            impute_strategy=scenario.get('impute_strategy', 'median'),
            balance_method=scenario['balance_method'],
            save_prefix=scenario['name']
        )
        
        all_results[scenario['name']] = results
    
    # Comparaison finale
    print("\n\n" + "="*70)
    print("COMPARAISON FINALE DES SCÉNARIOS")
    print("="*70)
    
    for scenario_name, results in all_results.items():
        print(f"\n{scenario_name}:")
        for model_name, metrics in results.items():
            print(f"  {model_name:20} - Test AUC: {metrics['test_metrics']['auc']:.4f} - F1: {metrics['test_metrics']['f1']:.4f}")



######################################################################
# SCÉNARIO : Imputation_Median_SMOTE
######################################################################

PIPELINE DE MODÉLISATION
Fichier : ../data/df_dropna_normalized.csv
Gestion des manquantes : impute
Stratégie d'imputation : median
Gestion du déséquilibre : smote
Chargement des données...


Dimensions : (19021, 39)
Valeurs manquantes : 2

Séparation des données...
Train : 15216 échantillons
Test : 3805 échantillons
Distribution train : Counter({0: 13473, 1: 1743})

Imputation avec méthode : median
Shape après imputation - Train: (15216, 32), Test: (3805, 32)

--- Gestion du déséquilibre : SMOTE ---
Distribution avant : Counter({0: 13473, 1: 1743})
Distribution après SMOTE : Counter({0: 13473, 1: 13473})

ENTRAÎNEMENT DES MODÈLES

KNN...
Temps d'entraînement : 0.01s

Random_Forest...
Temps d'entraînement : 22.11s

XGBoost...
Temps d'entraînement : 2.20s

ÉVALUATION DES MODÈLES

KNN

TRAIN:
  ACCURACY     : 0.8732
  PRECISION    : 0.8091
  RECALL       : 0.9769
  F1           : 0.8851
  AUC          : 0.9733

TEST:
  ACCURACY     : 0.6668
  PRECISION    : 0.1573
  RECALL       : 0.4381
  F1           : 0.2315
  AUC          : 0.5840

Matrice de confusion (Test):
[[2346 1023]
 [ 245  191]]

Random_Forest

TRAIN:
  ACCURACY     : 1.0000
  PRECISION    : 1.0000
  RECALL       

**TEST DU PIPELINE**

In [ ]:
# Exécution d'un seul scénario
trained_models, results = run_pipeline(
    filepath="../data/df_dropna_normalized.csv",
    target_column="embauche",
    missing_method="impute",      # "impute" ou "drop"
    impute_strategy="knn",     # "median", "mean", "knn"
    balance_method="weight",       # "none", "weight", "smote", "smote_tomek"
    save_prefix="model_knn_weight"
)


PIPELINE DE MODÉLISATION
Fichier : ../data/df_dropna_normalized.csv
Gestion des manquantes : impute
Stratégie d'imputation : knn
Gestion du déséquilibre : weight
Chargement des données...
Dimensions : (19021, 39)
Valeurs manquantes : 2

Séparation des données...
Train : 15216 échantillons
Test : 3805 échantillons
Distribution train : Counter({0: 13473, 1: 1743})

Imputation avec méthode : knn
Shape après imputation - Train: (15216, 32), Test: (3805, 32)

--- Gestion du déséquilibre : WEIGHT ---
Distribution avant : Counter({0: 13473, 1: 1743})
Poids calculés : {0: 0.5646849254063683, 1: 4.364888123924269}

ENTRAÎNEMENT DES MODÈLES

KNN...
Temps d'entraînement : 0.01s

Random_Forest...
Temps d'entraînement : 6.44s

XGBoost...
Temps d'entraînement : 0.51s

ÉVALUATION DES MODÈLES

KNN

TRAIN:
  ACCURACY     : 0.8928
  PRECISION    : 0.6739
  RECALL       : 0.1245
  F1           : 0.2102
  AUC          : 0.8728

TEST:
  ACCURACY     : 0.8699
  PRECISION    : 0.1609
  RECALL       : 0.0321